# ECSE-551 Mini Project 3
Authors:
* Ashley Meagher (260822930)
* Charles Sirois (261158513)  
* Emma ()

In [ ]:
# To specify where to load the data
in_colab = False
folder_path = 'drive/MyDrive/Colab Notebooks/ECSE 551_MP3'

%load_ext autoreload
%autoreload 2

# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as pltà

import pickle
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import torch

# Colab setup
if in_colab:
    from google.colab import drive
    from google.colab import data_table

    data_table.enable_dataframe_formatter()  # For interactive df viz

    drive.mount('/content/drive')

    #   # Import LogisticRegression
    #   import sys
    #   sys.path.insert(0, folder_path)
    #   from LogisticRegression import LogisticRegression

else:
    # Our imports
    # from LogisticRegression import LogisticRegression, cost_function
    ...